# AirBnB Barcelona: An analysis of listings



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./data/calendar.csv.gz', compression='gzip')
df.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,31380,2019-07-10,f,$75.00,$75.00,3,1125
1,18666,2019-07-10,f,$130.00,$130.00,3,730
2,18666,2019-07-11,f,$130.00,$130.00,3,730
3,18666,2019-07-12,f,$130.00,$130.00,3,730
4,18666,2019-07-13,f,$130.00,$130.00,3,730


In [5]:
df.shape

(7239045, 7)

In [11]:
df.describe()

,listing_id,minimum_nights,maximum_nights
count,7.239045e+06,7.239045e+06,7.239045e+06
mean,2.095877e+07,8.785608e+00,2.172749e+05
std,1.140940e+07,1.813377e+01,2.156397e+07
min,1.866600e+04,1.000000e+00,1.000000e+00
25%,1.176684e+07,1.000000e+00,6.000000e+01
50%,2.237577e+07,3.000000e+00,1.125000e+03
75%,3.156051e+07,4.000000e+00,1.125000e+03
max,3.658276e+07,9.000000e+02,2.147484e+09
